## I. Introduction
Organization of information is quintessential for efficient analysis workflows which use that information. For the ICARUS TPC subsystem, which has approximately 55,000 channels, this is especially true. Each channel has multiple characteristics and may be associated with higher level groupings (e.g. flat cables). SQLite provides highly portable database files and simple Python front-ends. This makes it an ideal tool for the storage and organization of this information. Additionally, the command line interface allows for quick access to the tables using standard SQL queries. 

## II. Setup
The basic tools for this demonstrator notebook are Pandas (for manipulating DataFrames/tables) and SQLite3 (for interfacing with database files). The SQLite3 library is provided as a standard library of Python. Additionally, it is necessary to set the name and path of the database file.

In [1]:
import pandas as pd
import sqlite3

DB_PATH = '/Users/mueller/Projects/channel_status/db/icarus_channels_dev.db'

## III. Table: Channel Info

This table stores channel-level information that relates the DAQ channel to its location in hardware.
* `channel_id` - The unique identifier for the channel.
* `tpc_number` - Specifies the TPC (0 = EE, 1 = EW, 2 = WE, 3 = WW).
* `plane_number` - The index of the plane that the channel belongs to (0 = Induction 1, 1 = Induction 2, 2 = Collection).
* `wire_number` - Indexes the channel within the current plane (and TPC)
* `slot_id` - The slot number of the readout board corresponding to the channel.
* `local_id` - The index of the channel within the board [0, 64).
* `group_id` - The index of the group of 32 channels.
* `fragment_id` - The decimal representation of the ArtDAQ fragment ID for the component.
* `flange_number` - The number of the flange [1, 20].
* `flange_name` - The geographical name of the flange.
* `cable_number` - The label of the flat cable that connects the TPC wire to the front-end.
* `channel_type` - The category of the channel (wired = channel has a physical wire, wireless = channel has no physical wire connected to the cable, ghost = channel not connected to a cable, and virtual = the slot where the channel would live has no readout board)

In [2]:
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql_query("SELECT * FROM channelinfo", conn)
conn.close()

display(df.head())

,channel_id,tpc_number,plane_number,wire_number,slot_id,local_id,group_id,fragment_id,flange_number,flange_name,cable_number,channel_type
0,0,0,0,0,0,63,0,4098,1,EE01M,C33,wired
1,1,0,0,1,0,62,0,4098,1,EE01M,C33,wired
2,2,0,0,2,0,61,0,4098,1,EE01M,C33,wired
3,3,0,0,3,0,60,0,4098,1,EE01M,C33,wired
4,4,0,0,4,0,59,0,4098,1,EE01M,C33,wired


## IV. Table: Flat Cables
The TPC flat cables bridge the gap between the wires and the readout electronics. They connect to the wires via the Comb-to-Cable interface boards and connect to the Decoupling and Biasing Boards (DBBs) on the front-end side. Each cable carries the signal from 32 adjacent wires of the same plane. The `cable_number` column in the `channelinfo` table directly maps to this table's `cable_number` column.

* `cable_number` - The unique identifier for the cable.
* `plane_type` - Specifies the plane corresponding to the cable.
* `crimp_style_at_ft` - The crimp/connector style at the feedthrough.
* `crimp_style_at_wire` - The crimp/connector style at the wire.
* `geometric_length` - The length bridged by the cable [mm].
* `cable_length_code` - The code used to identify the length specification of the cable.
* `real_length` - The real physical length of the cable [mm].
* `capacitance` - The capacitance of the cable [pF].

In [3]:
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql_query("SELECT * FROM flatcables", conn)
conn.close()

display(df.head())

,cable_number,plane_type,crimp_style_at_ft,crimp_style_at_wire,geometric_length,cable_length_code,real_length,capacitance
0,A01,IND1,A,A,3410.1,14,3780.0,177.66
1,A02,IND1,A,A,3482.2,14,3780.0,177.66
2,A03,IND1,A,A,3554.2,15,4050.0,190.35
3,A04,IND1,A,A,3626.3,15,4050.0,190.35
4,A05,IND1,A,A,3698.4,15,4050.0,190.35


In [33]:
import numpy as np
import pandas as pd
from database_utilities import parse_wire_entry, parse_map_entry

INPUT_PATH = '/Users/mueller/Projects/channel_status/inputs/'

lines = open(f'{INPUT_PATH}ICARUS-geometry.txt').readlines()
line_has_wire = ['C:' in x and 'T:' in x and 'P:' in x and 'W:' in x for x in lines]
wire_entries = np.array([parse_wire_entry(x) for xi, x in enumerate(lines) if line_has_wire[xi]])
wires = pd.DataFrame(wire_entries, columns=['c', 't', 'p', 'w', 'length', 'x', 'y0', 'z0', 'y1', 'z1'])

lines = open(f'{INPUT_PATH}ICARUS-channelmap.txt').readlines()
line_has_channel = ['C:' in x and 'T:' in x and 'P:' in x and 'W:' and '=>' in x in x for x in lines]
channel_entries = np.array([parse_map_entry(x) for xi, x in enumerate(lines) if line_has_channel[xi]])
logicalwires = pd.DataFrame(channel_entries, columns=['channel_id', 'c', 't', 'p', 'w'])
#vals = [tuple(x) for x in logicalwires[['channel_id', 'w', 'c', 't', 'p']].to_numpy()]

wires = wires.merge(logicalwires, left_on=['c', 't', 'p', 'w'], right_on=['c', 't', 'p', 'w'])
#display(wires.head())

def get_physical_wire_for_channel(g) -> tuple[float]:
    """
    Merges the logical wire information (assuming the input belongs to
    a single channel) into a single physical wire.

    Parameters
    ----------
    g: pd.DataFrame
        The DataFrame storing logical wire entries for the single
        channel.
    
    Returns
    -------
    x: float
        The x-coordinate of the physical wire.
    y0: float
        The y-coordinate of the first endpoint.
    z0: float
        The z-coordinate of the first endpoint.
    y1: float
        The y-coordinate of the second endpoint.
    z1: float
        The z-coordinate of the second endpoint.
    length: float
        The total length of the physical wire.
    capacitance: float
        The total capacitance of the physical wire.
    """
    endpoints = np.vstack([g[['y0', 'z0']].to_numpy(), g[['y1', 'z1']].to_numpy()]).astype(float)
    x = g['x'].astype(float).iloc[0]
    y0, z0 = endpoints[0,:]
    y1, z1 = endpoints[-1,:]
    length = np.sum(g['length'].astype(float))
    if g['p'].iloc[0] == 0:
        length = 942.0
    capacitance = length * (0.20 if g['p'].iloc[0] != 1 else 0.21)
    return x, y0, z0, y1, z1, length, capacitance

res = [(float(n), *get_physical_wire_for_channel(g)) for n, g in wires.groupby('channel_id')]

[('0', -358.73, -181.7, 0.0, -181.7, -894.951, 894.951, 178.99020000000002),
 ('1', -358.73, -181.4, 0.0, -181.4, -894.951, 894.951, 178.99020000000002),
 ('10', -358.73, -178.7, 0.0, -178.7, -894.951, 894.951, 178.99020000000002),
 ('100', -358.73, -151.7, 0.0, -151.7, -894.951, 894.951, 178.99020000000002),
 ('1000', -358.73, 118.3, 0.0, 118.3, -894.951, 894.951, 178.99020000000002),
 ('10000',
  -359.33,
  -181.85,
  -412.748,
  134.95,
  -229.843,
  365.809,
  73.16180000000001),
 ('10001',
  -359.33,
  -181.85,
  -412.401,
  134.95,
  -229.497,
  365.809,
  73.16180000000001),
 ('10002',
  -359.33,
  -181.85,
  -412.055,
  134.95,
  -229.15,
  365.809,
  73.16180000000001),
 ('10003',
  -359.33,
  -181.85,
  -411.708,
  134.95,
  -228.804,
  365.809,
  73.16180000000001),
 ('10004',
  -359.33,
  -181.85,
  -411.362,
  134.95,
  -228.458,
  365.809,
  73.16180000000001),
 ('10005',
  -359.33,
  -181.85,
  -411.016,
  134.95,
  -228.111,
  365.809,
  73.16180000000001),
 ('10006',
 